In [1]:
import os 
import shutil
from karabo_data import RunDirectory
from felpy.utils.daq_utils import load_data, shimadzu_reshape
from extra_data import open_run, RunDirectory, H5File
import numpy as np
from felpy.utils.os_utils import mkdir_p
from felpy.utils.vis_utils import  plot_fill_between, simple_line_plot, colorbar_plot
from felpy.analysis.optics.enclosed_energy import get_enclosed_energy
from felpy.analysis.optics.centroid import get_com
from labwork.about import dCache
from felpy.utils.np_utils import get_mesh
from felpy.utils.vis_utils import animate
from felpy.utils.vis_utils import plot_fill_between
from matplotlib import pyplot as plt
from felpy.analysis.optics.enclosed_energy import get_enclosed_energy
from tqdm import tqdm
from labwork.analysis.experiment.Diagnostics.shimadzu_tools import plot_com, plot_enclosed_energy, create_gif, plot_xgm


In [24]:
class Shimadzu:
    
    def __init__(self, proposal, run, VERBOSE = False):
        """
        a lazy class-type structure to deal with data collected by the shimadzu detector
        """
        self.proposal = proposal
        self.run = run

        self.sources = {}
        self.sources['upstream'] = "SA1_XTD9_IMGPII45/CAM/BEAMVIEW_SHIMADZU:daqOutput"
        self.sources['downstream'] = "SPB_EHD_HPVX2_1/CAM/CAMERA:daqOutput"
        
        self.VERBOSE = VERBOSE
        
    def load_upstream_data(self, px = 1, py = 1):
        
        try:
            upstream = open_run(self.proposal, self.run)

            self.u_ii = np.array(upstream.get_array(self.sources['upstream'], 'data.image.pixels'))
            self.u_ii = shimadzu_reshape(self.u_ii)[:,:,:,:]
            self.u_mesh = get_mesh(self.u_ii, px, py)

        except(KeyError):
            print("No upstream data available")
            
    def load_downstream_data(self, px = 1, py = 1):
        
        try:
            downstream = open_run(self.proposal, self.run)

            self.d_ii = np.array(downstream.get_array(self.sources['downstream'], 'data.image.pixels'))
            self.d_ii = shimadzu_reshape(self.d_ii)[:,:,:,:]
            self.d_mesh = get_mesh(self.d_ii, px, py)

        except(KeyError):
            print("No upstream data available")
            
    def create_gif(dataset, sdir, fname, title = "", xlabel = "x (pixels)", ylabel = "y (pixels)"):
        
        if dataset == 'upstream':
            ii = self.u_ii
            mesh = self.u_mesh
        elif dataset == 'downstream':
            ii = self.d_ii
            mesh = self.d_mesh
        
        
        
        plt.style.use(['science','ieee'])
        
        mkdir_p(sdir)

        tmp_dir = sdir + "/tmp/"
        mkdir_p(tmp_dir)

        for i in range(ii.shape[-2]):
            colorbar_plot(ii[:,:,i,:].mean(-1),
                         mesh, 
                         title = title,
                         xlabel = xlabel,
                         ylabel = ylabel,
                         clabel = "Intensity (a.u.)",
                         normalise = False,
                         context = 'talk',
                         cmap = 'jet',
                         label = "pulse: {}".format(i+1),
                         sdir = tmp_dir + "pulse_{:0>4d}".format(i))

        animate(tmp_dir, sdir, fname, delay = 0.1, rmdir = True)

In [26]:
s = Shimadzu('p002919', '97')
s.load_upstream_data()
s.load_downstream_data()
s.create_gif('upstream', dCache + "/tmp/", "test_goif")

UnboundLocalError: local variable 'ii' referenced before assignment